# How to query Azure Log Analytics in Jupyter

Based on [Use a Jupyter Notebook and kqlmagic extension to analyze data in Azure Data Explorer](https://docs.microsoft.com/en-us/azure/data-explorer/kqlmagic).

## Dependencies



In [ ]:
import sys
!{sys.executable} -m pip install Kqlmagic --no-cache-dir -q --upgrade
%reload_ext Kqlmagic

## Connect to the Azure Log Analytics workspace

* `workspace` parameter is **Workspace ID** of Azure Log Analytics Workspace resource
* When logging in for the first time, you will be asked to open Web browser and enter an authentication code (as for the [device code flow](https://docs.microsoft.com/en-us/azure/active-directory/develop/v2-oauth2-auth-code-flow)).

In [ ]:
%kql loganalytics://code;workspace='dad7ef45-e5f8-45af-b182-1d78ce0a5f71'

## Make a Query

[Querying Log Analytics using Kusto](https://docs.microsoft.com/en-us/azure/data-explorer/kusto/query/tutorial?pivots=azuremonitor) 

> Uncomment `%%capture` below if don't want to suppress query output

Query `AppTraces | where SeverityLevel == 3` retrieves all application traces (made with `logging` functions in Python) with severity level equal `3` (error).


In [ ]:
%%capture
%%kql 
AppTraces | where SeverityLevel == 3


## Convert results to Pandas

In [ ]:
df = _kql_raw_result_.to_dataframe()
df

## More comples query

All **errors** from successful Azure ML Runs

> Notice there is no `capture`, so the results are displayed in-place. You still can convert it to pandas from `_` or `_kql_raw_result_` variables.

In [ ]:
%%kql
AppTraces
| where Properties.level == "ERROR"
| extend ExperimentName = tostring(Properties.experiment_name), CorrelationId = tostring(Properties.correlation_id)
| join(AmlRunStatusChangedEvent
       | where Status == "Completed"
       | project RunId, RunStatus=Status, WorkspaceName) on $left.CorrelationId ==$right.RunId
